# Scraper

In [5]:
import pandas as pd 
import requests
import unicodedata
from tqdm import tqdm
from bs4 import BeautifulSoup

def get_links(): 
    response = requests.get('https://en.wikipedia.org/wiki/List_of_language_names')
    soup = BeautifulSoup(response.content, 'html.parser')
    links = soup.find_all('a')
    # links = [link.get('href') for link in links if (link.get('title') is not None) and ('language' in link.get('title'))]
    links = [link.get('href') for link in links if (link.get('title') is not None)]
    # links = [link for link in links if link.endswith('_language')]
    links = ["https://en.wikipedia.org"+link for link in links]
    return links

In [6]:
countries_and_territories = requests.get('https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_and_their_capitals_in_native_languages')
soup = BeautifulSoup(countries_and_territories.content, 'html.parser')

# how to specify multiple classes in find_all



table = soup.find('table', {'class': ['wikitable', 'sortable', 'jquery-tablesorter']})
regions = []
languages = []


table_rows = table.find_all('tr')
print(len(table_rows))

16


In [7]:
unesco_languages = pd.read_csv('Extinct languages - DATA SUMMARY.csv')
unesco_languages

,Name in English,Number of speakers,Degree of endangerment
0,South Italian,7500000,Vulnerable
1,Sicilian,5000000,Vulnerable
2,Low Saxon,4800000,Vulnerable
3,Belarusian,4000000,Vulnerable
4,Lombard,3500000,Definitely endangered
...,...,...,...
2534,|'Auni,0,Extinct
2535,|Xam,0,Extinct
2536,ǁKu ǁ'e,0,Extinct
2537,ǁKx'au,0,Extinct


In [8]:


def find_index_of_headers(url):
    response        = requests.get(url)
    content         = response.content
    parser          = BeautifulSoup(content, 'html.parser')
    table           = parser.find('table', {'class': 'infobox'})
    included_headers = {}
    
    for i in range(0, len(table.find_all('th', {'class': 'infobox-label'}))):
        included_headers[unicodedata.normalize('NFKD', table.find_all('th', {'class': 'infobox-label'}).__getitem__(i).getText())] = i
    return included_headers


urls = get_links()
print(len(urls))
print(urls[:10])

urls = list(set(urls))

urls = [url for url in urls if url not in ['Official_language', 'Minority_language']]
urls = [url for url in urls if url.startswith('List_of_countries') == False]
urls = [url for url in urls if url.count(':') == 1]
urls = [url for url in urls if '?' not in url]

3540
['https://en.wikipedia.org/wiki/Main_Page', 'https://en.wikipedia.org/wiki/Wikipedia:Contents', 'https://en.wikipedia.org/wiki/Portal:Current_events', 'https://en.wikipedia.org/wiki/Special:Random', 'https://en.wikipedia.org/wiki/Wikipedia:About', 'https://en.wikipedia.org//en.wikipedia.org/wiki/Wikipedia:Contact_us', 'https://en.wikipedia.orghttps://donate.wikimedia.org/wiki/Special:FundraiserRedirector?utm_source=donate&utm_medium=sidebar&utm_campaign=C13_en.wikipedia.org&uselang=en', 'https://en.wikipedia.org/wiki/Help:Contents', 'https://en.wikipedia.org/wiki/Help:Introduction', 'https://en.wikipedia.org/wiki/Wikipedia:Community_portal']


In [9]:
urls = urls[0:100]

In [10]:
data = []
errors = []

for url in tqdm(urls): 
    # try: 
        lang = url.split('/')[-1]

        # print(requests.get(url).status_code)
        # print(url)
        response        = requests.get(url)
        content         = response.content
        parser          = BeautifulSoup(content, 'html.parser')
        table           = parser.find('table', {'class': 'infobox'})

        if table == None:
             print("No table found ", url)
             continue

        header_indices  = find_index_of_headers(url)

        # print(header_indices)

        infobox_data = table.find_all('td', {'class': 'infobox-data'})

        off_lang        = infobox_data[header_indices['Official language in']].text if 'Official language in' in header_indices else None
        rec_min_lang    = infobox_data[header_indices['Recognised minoritylanguage in']].text if 'Recognised minoritylanguage in' in header_indices else None
        # speakers        = infobox_data[header_indices['Speakers']].text             if 'Speakers' in header_indices else None
        iso3code        = infobox_data[header_indices['ISO 639-3']].text            if 'ISO 639-3' in header_indices else None

        if 'Speakers' in header_indices.keys():
            speakers = infobox_data[header_indices['Speakers']].text
        elif 'Native speakers' in header_indices.keys():
            speakers = infobox_data[header_indices['Native speakers']].text
        else:
            speakers = None

        if 'Region' in header_indices.keys():
            regions = infobox_data[header_indices['Region']].text
        elif 'Native Region' in header_indices.keys():
            regions = infobox_data[header_indices['Native Region']].text
        else:
            regions = None

        family          = [x for x in infobox_data[header_indices['Language family']].get_text('title').split('title') if x != "\n"]\
                            if 'Language family' in header_indices else None
        
        dialects        = [x for x in infobox_data[header_indices['Dialects']].get_text('title').split('title') if x != "\n"] \
                            if 'Dialects' in header_indices else None
        
        # print('Regions: ', regions)
        data.append((lang, family, dialects, iso3code, speakers, regions, off_lang, rec_min_lang))
    # except Exception as e: 
    #     errors.append((e, url))
    #     continue

df = pd.DataFrame(data, columns=['lang', 'family', 'dialects', 'iso3code', 'speakers', 'regions', 'off_lang', 'rec_min_lang'])


 22%|██▏       | 22/100 [00:37<01:48,  1.38s/it]

No table found  https://en.wikipedia.org/wiki/List_of_countries_where_Spanish_is_an_official_language


 28%|██▊       | 28/100 [00:51<01:54,  1.59s/it]

No table found  https://en.wikipedia.org/wiki/Provinces_of_Costa_Rica


 35%|███▌      | 35/100 [01:08<02:21,  2.17s/it]

In [63]:
df.to_csv('wiki_languages_new.csv')

In [6]:
languages = df

In [71]:
df = pd.read_csv('wiki_languages.csv')
languages = df

In [72]:
cols = languages.columns[1:]
for col in cols: 
    languages[col] = languages[col].apply(lambda x: [] if pd.isnull(x) else [x])
    languages[col] = languages[col].apply(lambda values: [x[:x.find('[')] if '[' in x else x for x in values])

In [80]:
languages[languages['lang'].apply(lambda x: 'punjabi' in x.lower())]

,lang,family,dialects,iso3code,speakers,regions,off_lang,rec_min_lang
196,Punjabi_language,[],[],[Either:pan – Panjabipnb – Western Panjabi],[],[Punjab],[ Pakistan\n Punjab (provincial)],[]


In [8]:
languages.isna().sum().sort_values()/len(languages)

lang            0.000000
iso3code        0.583333
family          0.593750
regions         0.645833
speakers        0.666667
off_lang        0.822917
dialects        0.843750
rec_min_lang    0.864583
dtype: float64

In [9]:
languages[:20]

,lang,family,dialects,iso3code,speakers,regions,off_lang,rec_min_lang
0,Aguacateco_language,"[Mayan, \nQuichean–Mamean, Greater Mamean, Ixi...",None,agu,"10,100 in Guatemala (2019 census)[1]20 in Mexi...",Huehuetenango,None,Guatemala\n Mexico
1,Archi_language,"[Northeast Caucasian, Lezgic, [2], Archi]",None,aqc,970 (2010 census)[1],"Archib, Dagestan",None,None
2,Republic_of_Artsakh,None,None,None,None,None,None,None
3,Mari_El,None,None,None,None,None,None,None
4,Sumerian_language,[Language isolate],"[Emesal, Emesisa, [1], Emetena, [1], Emesukudd...",sux,None,Mesopotamia (modern-day Iraq),None,None
5,Sinhala_language,"[Indo-European, Indo-Iranian, Indo-Aryan, Sout...","[Vedda, (perhaps a , Creole language, )]",sin,L1: 16 million (2021)[1]L2: 2.0 million (1997...,None,Sri Lanka,None
6,Baybayin,None,None,None,None,None,None,None
7,Volap%C3%BCk,None,None,vol,None,None,None,None
8,Arvanitika,"[Indo-European, Albanian, Tosk, Arvanitika]","[Viotia Arvanitika, Attiki Arvanitika, Salamin...",aat,"50,000 (2007)[3](may be republished older data)","Attica, Boeotia, South Euboea, Saronic Islands...",None,None
9,Guam,None,None,None,None,None,None,None


In [10]:
unesco_languages

,Name in English,Number of speakers,Degree of endangerment
0,South Italian,7500000,Vulnerable
1,Sicilian,5000000,Vulnerable
2,Low Saxon,4800000,Vulnerable
3,Belarusian,4000000,Vulnerable
4,Lombard,3500000,Definitely endangered
...,...,...,...
2534,|'Auni,0,Extinct
2535,|Xam,0,Extinct
2536,ǁKu ǁ'e,0,Extinct
2537,ǁKx'au,0,Extinct


In [35]:
languages = languages.merge(unesco_languages, left_on='lang', right_on='Name in English')
print(languages.columns)
languages

AttributeError: 'list' object has no attribute 'merge'